In [ ]:
import csaf_f16.ngoals as f16g
from csaf_f16.acas import AcasScenarioViewer

import re
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

## Scenario 1: Rejoin

### Summary

Ownship is attempting to fly north at a constant airspeed, while intruder is attempting a eastward turn while position to west of the intruder. Both aircraft start at the same north  / south position (flying in formation).


### Coordinates

* **Altitude** - Discrete - {15k, 35k} ft - Both aircraft are positioned at the same altitude
* **Airspeed** - Discrete - {600, 1k} ft/s - The ownship is given this airspeed
* **Relative East / West Distance** - Continuous - [7k, 10k] ft - Intruder is placed at the origin and intruder is placed at (0, relative distance)
* **Relative Intruder Airspeed** - Continuous - [-150, 300] ft/s - Ownship is giving the above airspeed and intruder is given ownship's airspeed + relative value
* **Relative Intruder Heading Angle** - Continuous - [-pi, pi] rad - Ownship is facing North and intruder is given this heading angle

In [ ]:
ar = f16g.AcasRejoinScenario()
sys = ar.generate_system((15E3, 1000.0, 12000.0, 0.0, 0.0))
trajs = sys.simulate_tspan((0.0, 20.0))
av = AcasScenarioViewer(trajs, ar)

In [ ]:
f, ax = av.summary_plot(bounds = ((-3000, 12000), (-1000, 21000)))
ax.annotate(s='', xy=(0.0,0.0), xytext=(12000.0,0), arrowprops=dict(arrowstyle='<->'))
ax.annotate(s='horizontal distance', xy=(6000.0,500.0), ha='center')

## Scenario 2: Airport

### Summary

Ownship and intruder are turning into one another. The ownship has ACAS-Xu autopilot, while the intruder does not. Both aircraft start at the same North  / South position.

### Coordinates

* **Altitude** - Discrete - {15k, 35k} ft - Both aircraft are positioned at the same altitude
* **Airspeed** - Discrete - {600, 1k} ft/s - The ownship is given this airspeed
* **Relative East / West Distance** - Continuous - [7k, 10k] ft - Intruder is placed at the origin and intruder is placed at (0, relative distance)
* **Relative Intruder Airspeed** - Continuous - [-150, 300] ft/s - Ownship is giving the above airspeed and intruder is given ownship's airspeed + relative value
* **Relative Intruder Heading Angle** - Continuous - [-pi, pi] rad - Ownship is facing North and intruder is given this heading angle

In [ ]:
ar = f16g.AcasAirportScenario()
sys = ar.generate_system((15E3, 1000.0, 12000.0, 0.0, 0.0))
trajs = sys.simulate_tspan((0.0, 20.0))
av = AcasScenarioViewer(trajs, ar)

In [ ]:
f, ax = av.summary_plot(bounds = ((-6000, 6000), (-1000, 21000)))
ax.annotate(s='', xy=(-6000.0,0.0), xytext=(6000.0,0.0), arrowprops=dict(arrowstyle='<->'))
ax.annotate(s='horizontal distance', xy=(0.0,500.0), ha='center')

## Scenario 3: Head On

### Summary

Both aircraft are at the same east / west position and can be positioned head on. Both attempt to fly straight at a constant airspeed.

### Coordinates

* **Altitude** - Discrete - {15k, 35k} ft - Both aircraft are positioned at the same altitude
* **Airspeed** - Discrete - {600, 1k} ft/s - The ownship is given this airspeed
* **Relative North / South Distance** - Continuous - [7k, 10k] ft - Intruder is placed at the origin and intruder is placed at (relative distance, 0)
* **Relative Intruder Airspeed** - Continuous - [-150, 300] ft/s - Ownship is giving the above airspeed and intruder is given ownship's airspeed + relative value
* **Relative Intruder Heading Angle** - Continuous - [-pi, pi] rad - Ownship is facing North and intruder is given this heading angle

In [ ]:
ar = f16g.AcasHeadOnScenario()
sys = ar.generate_system((15E3, 1000.0, 12000.0, 0.0, -3.141592))
trajs = sys.simulate_tspan((0.0, 10.0))
av = AcasScenarioViewer(trajs, ar)

In [ ]:
f, ax = av.summary_plot(bounds = ((-3000, 3000), (-1000, 14000)))
ax.annotate(s='', xy=(-1000.0,0.0), xytext=(-1000.0,12000.0), arrowprops=dict(arrowstyle='<->'))
ax.annotate(s='vertical distance', xy=(-2500.0,6000.0), ha='center')

## Goal Falsification

Constrained Bayesion optimization is used to attempt to falsify the 250 ft well clear region.

#### Constraints

* minimum distance to the intruder is set to 7000 ft.
* the airspeed of the intruder is constrained to be "reasonable" (meaning not to trip a simulation error). This works out to be between [550, 1100] ft/s.

#### Kernel

* an standard periodic kernel is used with ARD turned on to created different lengthscales. 20 ft/s is used for speed quantities, 200 ft is used for distances, and 0.01 rad is used for angular.

In [ ]:
def video_title(scen):
    """generate a filename for a scenario object"""
    scen_sc = re.sub('(?!^)([A-Z]+)', r'_\1',scen.__class__.__name__).lower()
    return f"{scen_sc}_{datetime.now().strftime('%m_%d_%H_%M')}"


In [ ]:
#g = f16g.AcasRejoin()
#g = f16g.AcasHeadOnGoal()
g = f16g.AcasAirportGoal()

In [ ]:
g.test(max_time=60*5)

In [ ]:
# plot GPyOpt's Convergence Information
g.optimizer.plot_convergence()

In [ ]:
# run the best example and plot the results (using AcasScenarioViewer)
from csaf_f16.acas import AcasScenarioViewer

x0 = g.optimizer.x_opt
scen = g.scenario_type()
trajs, p = g.run_sim(x0)

viewer = AcasScenarioViewer(trajs, scen)
fig, ax = viewer.summary_plot()
ax.set_xlim(-10000, 10000)
ax.set_ylim(0, 25000)
plt.show()

In [ ]:
g.optimizer.x_opt

In [ ]:
#from IPython.display import HTML
#from matplotlib import animation
#viewer = AcasScenarioViewer(trajs, ar)
#anim = viewer.summary_video(bounds=((-15000, 15000), (-5000, 25000)), msize=0.0005)
# use this to render a video player in the notebooks
#HTML(anim.to_jshtml())
# use this to write it as gif to cwd
#writervideo = animation.FFMpegWriter(fps=30) 
#anim.save(f"{video_title(g.scenario_type())}.gif", writer=writervideo)

In [ ]:
import numpy as np
sidxs= np.argsort(g.optimizer.Y.flatten())
Xt, Yt = g.optimizer.X[sidxs[:10]], g.optimizer.Y[sidxs[:10]]

In [ ]:
for idx, xi in enumerate(Xt):
    print("-----")
    print(idx, xi)
    scen = g.scenario_type()
    trajs, p = g.run_sim(xi)

    viewer = AcasScenarioViewer(trajs, scen)
    fig, ax = viewer.summary_plot()
    ax.set_xlim(-10000, 10000)
    ax.set_ylim(0, 25000)
    plt.show()
    print("-----")